In [1]:
import pymongo

# 连接MongoDB
MongoClient(host= , port= )

In [46]:
client = pymongo.MongoClient(host='localhost',port=27017)
# client = pymongo.MongoClient('mongodb://localhost:27017/')

# 指定数据库

In [81]:
db = client.test
db = client['test'] # 或者

# 指定集合
MongoDB的每个数据库又包含许多集合（collection），它们类似于关系型数据库中的表。

In [82]:
collection = db.students
collection = db['students'] # 或者

# 插入数据

In [83]:
student1 = {'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'} # 字典形式
student2 = {'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}
result = collection.insert([student1, student2])
print(result)
'''每条数据其实都有一个_id属性来唯一标识, insert()方法会在执行后返回_id值。''';

[ObjectId('6163b7194d26116f614ffe01'), ObjectId('6163b7194d26116f614ffe02')]


c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


在PyMongo中，官方已经不推荐使用insert方法了。目前，官方推荐使用insert_one和insert_many方法来分别插入单条记录和多条记录。
### insert_one方法

In [84]:
student = {'id': '20170101','name': 'Jack','age': 20,'gender': 'male'}
result = collection.insert_one(student)
print(result) # 返回的是 InsertOneResult 对象
print(result.inserted_id) # 调用其 inserted_id  属性获取 _id 

6163b71a4d26116f614ffe03


### insert_many方法

In [85]:
student1 = {'id': '20170101', 'name': 'Bob', 'age': 20, 'gender': 'male'} 
student2 = {'id': '20170202', 'name': 'Tim', 'age': 21, 'gender': 'male'}
result = collection.insert_many([student1, student2])
print(result) # 返回的是 InsertOneResult 对象
print(result.inserted_ids) # 调用其 inserted_ids  属性获取 _id 

[ObjectId('6163b71c4d26116f614ffe04'), ObjectId('6163b71c4d26116f614ffe05')]


# 查询
### find_one查询 得到的是单个结果

In [52]:
result = collection.find_one({'name':'Mike'})
print('类型：',type(result))
print('结果：',result)

类型： <class 'dict'>
结果： {'_id': ObjectId('6163b5eb4d26116f614ffdfd'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


### find查询 则返回一个生成器对象

In [70]:
results = collection.find({'age': 20})
print('返回一个生成器对象:',results)
for result in results:
    print(result)

返回一个生成器对象: <pymongo.cursor.Cursor object at 0x000000000A4A8B48>


+ 查找年龄大于20的所有学生

In [79]:
results = collection.find({'age': {'$gt': 20}})
print('返回一个生成器对象:',results)
for result in results:
    print(result)

返回一个生成器对象: <pymongo.cursor.Cursor object at 0x000000000A7FBF88>


这里查询的条件键值已经不是单纯的数字了，而是一个字典，其键名为比较符号 $gt ，意思是大于，键值为 20 。
**比较符号表：**

|符号|含义|示例|
|:---:|:---:|:---:|
|\$lt|小于|{'age' : {'\$lt':20}}|
|\$gt|大于|{'age' : {'\$gt':20}}|
|\$lte|小于等于|{'age' : {'\$lte':20}}|
|\$gte|大于等于|{'age' : {'\$gte':20}}|
|\$ne|不等于|{'age' : {'\$ne':20}}|
|\$in|在范围内|{'age' : {'\$in':[20,30]}}|
|\$nin|不在范围内|{'age' : {'\$nin':[20,30]}}|

+ 查询名字以 'J' 开头的学生数据

In [58]:
results = collection.find({'name': {'$regex': '^J.*'}})
print('返回一个生成器对象:',results)
for result in results:
    print(result)

返回一个生成器对象: <pymongo.cursor.Cursor object at 0x000000000A8069C8>
{'_id': ObjectId('6163b5eb4d26116f614ffdfc'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('6163b5ed4d26116f614ffdfe'), 'id': '20170101', 'name': 'Jack', 'age': 20, 'gender': 'male'}


<center><font face="黑体" size=5>其他功能符号:</font></center>

|符号|含义|示例|示例含义|
|:---:|:---:|:---:|:---:|
|\$regex|匹配正则表达式|{'name' : {'\$regex':'^M.*'}}|name以M开头|
|\$exists|属性是否存在|{'name' : {'\$exists':True}}|name属性存在|
|\$type|类型判断|{'age' : {'\$type':'int'}}|age的类型为int|
|\$mod|数字模操作|{'age' : {'\$mod':[5,0]}}|年龄模5余0|
|\$text|文本查询|{'\$text' : {'\$search':'Mike'}}|text类型的属性中包含Mike字符串|
|\$where|高级条件查询|{'\where' : 'obj.fans_count == obj.follows_count'}|自身粉丝数等于关注数|

# 计数

In [59]:
total = collection.find().count()
num = collection.find({'age':{'$gt':20}}).count()
print(total, num)

5 2


c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.
c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


# 排序
 pymongo.ASCENDING  指定升序。pymongo.DESCENDING 指定降序。

In [60]:
results = collection.find().sort('name', pymongo.ASCENDING)
print([result['name'] for result in results])

['Bob', 'Jack', 'Jordan', 'Mike', 'Tim']


# 偏移
如果只需某几个元素，可以利用skip()方法偏移几个位置，比如偏移2,代表忽略前两个元素，得到第3个及以后的元素：

In [61]:
results = collection.find().sort('name', pymongo.ASCENDING).skip(2)
print([result['name'] for result in results])

['Jordan', 'Mike', 'Tim']


limit()方法指定要取的结果个数

In [63]:
results = collection.find().sort('name', pymongo.ASCENDING).skip(2).limit(2)
print([result['name'] for result in results])

['Jordan', 'Mike']


# 更新
使用 update(条件，数据)，指定更新的条件和更新后的数据。

In [67]:
condition = {'name': 'Mike'} # 设置条件
student = collection.find_one(condition) # 找到
student['age'] = 25 # 更改
result = collection.update(condition, student) # 替换
print(result)
'''返回结果是字典形式， ok  代表执行成功， nModified  代表影响的数据条数。''';

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}


c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.


+ update_one

In [68]:
condition = {'name': 'Mike'}
student = collection.find_one(condition)
student['age'] = 26
result = collection.update_one(condition, {'$set': student})
print(result)
print(result.matched_count, result.modified_count)
'''其返回结果是UpdateResult类型。然后分别调用matched_count和modified_count属性，
可以获得匹配的数据条数和影响的数据条数。''';

1 1


+ update_many

In [72]:
condition = {'age': {'$gt': 18}} # 查询条件为年龄大于 20
result = collection.update_many(condition, {'$inc': {'age': 1}}) # 更新条件为年龄加 1
print(result)
print(result.matched_count, result.modified_count)

5 5


# 删除
remove(条件)

In [73]:
result = collection.remove({'name': 'Jordan'})
print(result)

{'n': 1, 'ok': 1.0}


c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


+ delete_one(条件)

In [76]:
result = collection.delete_one({'name': 'Mike'})

print(result.deleted_count)

0


+ delete_many(条件)

In [88]:
result = collection.delete_many({'age': {'$lt': 25}})
print(result)
print(result.deleted_count)

0


+ 全部删除

In [87]:
result = collection.remove()
print(result)

{'n': 0, 'ok': 1.0}


c:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


# 其他操作  
另外， PyMongo  还提供了一些组合方法，如 find_one_and_delete 、 find_one_and_replace  和 find_one_and_update ，它们分别用于查找后删除、替换和更新操作，其使用方法与上述方法基本一致。  
另外，我们还可以对索引进行操作，相关方法有 create_index 、 create_indexes  和 drop_index  等。